In [17]:
import sys
import os

import pandas as pd
import numpy as np
import skvideo.io
import tensorflow as tf


dir_path = os.getcwd()

train_folder = os.path.join(dir_path,"data/train/")
test_folder = os.path.join(dir_path,"data/test/")

train_target = os.path.join(dir_path,'data/train_target.csv')

print("Current dir -> ", dir_path)
print("Train folder -> ",train_folder)
print("Train target -> ",train_target)
print("Test folder -> ",test_folder)


def get_videos_from_folder(data_folder):
    '''
    get a list of video x wehre each video is a numpy array in the format [n_frames,width,height] 
    with uint8 elements.
    argument: relative path to the data_folder from the source folder.
    '''
    data_folder = os.path.join(dir_path,data_folder)
    x = []
    file_names = []
    
    if os.path.isdir(data_folder):
        for dirpath, dirnames, filenames in os.walk(data_folder):
            for filename in filenames:
                file_path = os.path.join(dirpath, filename)
                statinfo = os.stat(file_path)
                if statinfo.st_size != 0:
                    video = skvideo.io.vread(file_path, outputdict={"-pix_fmt": "gray"})[:, :, :, 0]
                    x.append(video)
                    file_names.append(int(filename.split(".")[0]))

    indices = sorted(range(len(file_names)), key=file_names.__getitem__)
    x = np.take(x,indices)
    return x

def get_target_from_csv(csv_file):
    '''
    get a numpy array y of labels. the order follows the id of video. 
    argument: relative path to the csv_file from the source folder.
    '''
    csv_file = os.path.join(dir_path,csv_file)
    with open(csv_file, 'r') as csvfile:
        label_reader = pd.read_csv(csvfile)
        y = label_reader['y']
        
    y = np.array(y)
    return y


def save_solution(csv_file,prob_positive_class):
    with open(csv_file, 'w') as csv:
        df = pd.DataFrame.from_dict({'id':range(len(prob_positive_class)),'y': prob_positive_class})
        df.to_csv(csv,index = False)


Current dir ->  /home/francesco/Scrivania/AML-18/task4
Train folder ->  /home/francesco/Scrivania/AML-18/task4/data/train/
Train target ->  /home/francesco/Scrivania/AML-18/task4/data/train_target.csv
Test folder ->  /home/francesco/Scrivania/AML-18/task4/data/test/


In [18]:
def _int64_feature(value):
    return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))

def _bytes_feature(value):
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

def save_tf_record(x,file_name,y = None):
    writer = tf.python_io.TFRecordWriter(file_name)
    if y is None:
        for video in x:
            sys.stdout.flush()
            feature = {'len': _int64_feature(video.shape[0]),
                       'height': _int64_feature(video.shape[1]),
                       'width': _int64_feature(video.shape[2]),
                       'video': _bytes_feature(tf.compat.as_bytes(video.tostring()))}
            example = tf.train.Example(features=tf.train.Features(feature=feature))
            writer.write(example.SerializeToString())
    else:
        for video,label in zip(x,y):
            sys.stdout.flush()
            feature = {'len': _int64_feature(video.shape[0]),
                       'height': _int64_feature(video.shape[1]),
                       'width': _int64_feature(video.shape[2]),
                       'video': _bytes_feature(tf.compat.as_bytes(video.tostring())),
                       'label': _int64_feature(label)}
            example = tf.train.Example(features=tf.train.Features(feature=feature))
            writer.write(example.SerializeToString())
    
    writer.close()
    sys.stdout.flush()

def prob_positive_class_from_prediction(pred):
    return np.array([p['probabilities'][1] for p in pred])

def decode(serialized_example):
    features = tf.parse_single_example(
        serialized_example,
        features={
            'len': tf.FixedLenFeature([], tf.int64),
            'height': tf.FixedLenFeature([], tf.int64),
            'width': tf.FixedLenFeature([], tf.int64),
            'label': tf.FixedLenFeature([], tf.int64,default_value = 0),
            'video': tf.FixedLenFeature([], tf.string),
        })
    video = tf.decode_raw(features['video'], tf.uint8)
    height = features['height']
    width = features['width']
    length = features['len']
    shape = tf.stack([length,height,width])
    video = tf.reshape(video,shape)
    label = features['label']
    features = {'video':video}
    return features,label

def input_fn_from_dataset(files,batch_size = 1,num_epochs = None,shuffle = True):
    data_set = tf.data.TFRecordDataset(files)
    if shuffle:
        data_set = data_set.shuffle(buffer_size=len(files)) 
    data_set = data_set.map(decode)
    data_set = data_set.padded_batch(batch_size,padded_shapes= ({'video':[212,100,100]},[]))
    data_set = data_set.repeat(num_epochs)
    data_set = data_set.prefetch(batch_size)
    
    return data_set

def decode_frame(serialized_example):
    features = tf.parse_single_example(
        serialized_example,
        features={
            'len': tf.FixedLenFeature([], tf.int64),
            'height': tf.FixedLenFeature([], tf.int64),
            'width': tf.FixedLenFeature([], tf.int64),
            'label': tf.FixedLenFeature([], tf.int64,default_value = 0),
            'video': tf.FixedLenFeature([], tf.string),
        })
    video = tf.decode_raw(features['video'], tf.uint8)
    height = features['height']
    width = features['width']
    length = features['len']
    shape = tf.stack([length,height,length])
    video = tf.reshape(video,shape)
    label = features['label']
    label = tf.expand_dims(label,axis=-1)
    label = tf.tile(label,tf.expand_dims(length,axis=-1))
    features = {'frame':video}
    return features,label

def input_fn_frame_from_dataset(files,batch_size = 1,num_epochs = None):
    data_set = tf.data.TFRecordDataset(files)
    data_set = data_set.shuffle(buffer_size=len(files)) 
    data_set = data_set.map(decode_frame)
    data_set = data_set.shuffle(buffer_size=batch_size)
    data_set = data_set.apply(tf.contrib.data.unbatch())
    data_set = data_set.batch(batch_size)
    data_set = data_set.repeat(num_epochs)
    data_set = data_set.prefetch(batch_size)
    
    return data_set

In [32]:
my_solution_file = os.path.join(dir_path,'data/solution.csv')

tf_record_dir = os.path.join(dir_path,'tf_records')
os.makedirs(tf_record_dir, exist_ok=True)

tf_record_train = os.path.join(tf_record_dir, 'train' + '.tfrecords')
tf_record_test = os.path.join(tf_record_dir, 'test' + '.tfrecords')

if not os.path.exists(tf_record_train):
    x_train = get_videos_from_folder(train_folder)
    y_train = get_target_from_csv(train_target)
    save_tf_record(x_train,tf_record_train,y = y_train)

if not os.path.exists(tf_record_test):
    x_test = get_videos_from_folder(test_folder)
    save_tf_record(x_test,tf_record_test)

feature_col_video = tf.feature_column.numeric_column('video',shape=[212,100,100],dtype=tf.uint8)
feature_col_frame = tf.feature_column.numeric_column('frame',shape = [100,100],dtype=tf.uint8)
batchsize_video = 1

print("Here")
print(feature_col_frame)
print(feature_col_video)
print(feature_col_video[1])

estimator = tf.estimator.LinearClassifier(feature_columns = [feature_col_video],model_dir='tf_checkpoints')
estimator.train(input_fn=lambda: input_fn_from_dataset(tf_record_train, batch_size=batchsize_video),max_steps=1)
pred = estimator.predict(input_fn=lambda: input_fn_from_dataset(tf_record_test, batch_size=batchsize_video,num_epochs=1,shuffle = False))
dummy_solution = prob_positive_class_from_prediction(pred)
save_solution(my_solution_file,dummy_solution)

Here
_NumericColumn(key='frame', shape=(100, 100), default_value=None, dtype=tf.uint8, normalizer_fn=None)
_NumericColumn(key='video', shape=(212, 100, 100), default_value=None, dtype=tf.uint8, normalizer_fn=None)
(212, 100, 100)
INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'tf_checkpoints', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f9e23a0a160>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evalua